Hotel Scraping project

In [1]:
!pip install selenium beautifulsoup4 webdriver_manager requests lxml


In [2]:
from typing import *

In [3]:
from datetime import datetime,timedelta

def build_url(time_to_travel: int, length_of_stay: int) -> str:
    today = datetime.today()

    checkin_date = today + timedelta(days=time_to_travel)
    checkout_date = checkin_date + timedelta(days=length_of_stay)
    checkin_date_str = checkin_date.strftime('%Y-%m-%d')
    checkout_date_str = checkout_date.strftime('%Y-%m-%d')
    url = f'https://www.booking.com/searchresults.en-gb.html?ss=New+York&ssne=New+York&ssne_untouched=New+York&lang=en-gb&dest_id=20088325&dest_type=city&checkin={checkin_date_str}&checkout={checkout_date_str}&group_adults=2&no_rooms=1&group_children=0&selected_currency=USD'
    return url

### Interaction functions

#### Genius Modal closer
There is a genius modal that sometimes pops up, and we need to close it

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException


def close_genius_modal(driver: webdriver.Chrome) -> bool:
    try:
        genius_modal_close_button = driver.find_element(By.XPATH, '//button[@aria-label="Dismiss sign in information."]')
        genius_modal_close_button.click()
        return True 
    except (NoSuchElementException, TimeoutException):
        return False

#### Load More 
We are mimiccing a real user interaction by going up and down in the page with the `HOME` and `END` keys. when we cant load more results in this method, we click on the load more results button

In [5]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
def load_more(driver: webdriver.Chrome) -> bool:
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.HOME)
    time.sleep(0.5)
    body.send_keys(Keys.END)

    try:
        # Scroll down to attempt to load more results
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        time.sleep(1)  # Allow new elements to load
        
        # Find the 'Load more results' button
        button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Load more results')]]"))
        )
        
        # Scroll directly to the button before clicking (ensures visibility)
        ActionChains(driver).move_to_element(button).perform()
        button.click()
        
        return True  # Successfully clicked the button

    except Exception:
        return False  # No button found or unable to click


### Mimic a real-user's driver
the Booking.com site can detect if a bot is trying to access it, and it will give it a different type of html, not the one you can see if you try to open the chrome devtools. In order to overcome this problem, and to be able to run the scraping in parrallel, we needed to define the driver in such way, that it will be headless and also behave like a user used site 

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    
    if headless:
        options.add_argument("--headless=new")  # Ensures modern headless mode
        options.add_argument("--disable-gpu")  # Fixes rendering issues
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
    
    # 🚀 Make the browser appear more human-like
    options.add_argument("--window-size=1920,1080")  # Standard screen size
    options.add_argument("--start-maximized")  # Maximize on launch
    options.add_argument("--disable-blink-features=AutomationControlled")  # Prevent detection
    options.add_experimental_option("excludeSwitches", ["enable-automation"])  # Prevent 'bot' flag
    options.add_experimental_option("useAutomationExtension", False)
    
    # Change User-Agent to a normal browser
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Remove webdriver property to avoid detection
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    return driver

In [7]:
import bs4
from bs4 import BeautifulSoup


def get_hotels(driver: webdriver.Chrome, url: str) -> List[bs4.element.Tag]:
    driver.get(url)

    soup = None
    hotels=[]
    genius_modal_closed = False
    previous_hotel_count = 0
    while len(hotels) < 100:
        if not genius_modal_closed:
            time.sleep(6)
            if close_genius_modal(driver):
                genius_modal_closed = True

        load_more(driver)
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        hotels = soup.find_all('div', {'data-testid': "property-card"})
        
        if len(hotels) == previous_hotel_count:
            print("No new hotels found")
            break
        previous_hotel_count = len(hotels)
        
    return hotels[:100]

### Extract features into DataFrame

In [8]:
def extract_name(hotel: bs4.element.Tag) -> Dict[str, str]:    
    return {
        'name': hotel.find('div', {'data-testid': 'title'}).text,
    }

In [9]:
import re
from lxml import etree

def extract_price(hotel: bs4.element.Tag) -> Dict[str, Optional[int]]:
    price = {
        'original_price': None,
        'discounted_price': None,
        'discount': None 
    }
    
    hotel_tree = etree.HTML(str(hotel))
    price_div_text_res = hotel_tree.xpath('//div[@data-testid="taxes-and-charges"]/preceding-sibling::div[1]/text()')
    
    if len(price_div_text_res) > 0:
        price_text = price_div_text_res[0]
        
        pattern_format1 = r"Original price US\$(\d{1,3}(?:,\d{3})*)\. Current price US\$(\d{1,3}(?:,\d{3})*)"

        # Regex pattern to match format 2 (Single Price)
        pattern_format2 = r"Price US\$(\d{1,3}(?:,\d{3})*)"

        # Try to match format 1 (discounted price case)
        match1 = re.search(pattern_format1, price_text)
        if match1:
            n1 = int(match1.group(1).replace(",", ""))  # Convert to integer
            n2 = int(match1.group(2).replace(",", ""))
            return {'original_price': n1, 'discounted_price': n2, 'discount': True}

        # Try to match format 2 (single price case)
        match2 = re.search(pattern_format2, price_text)
        if match2:
            n3 = int(match2.group(1).replace(",", ""))
            return {'original_price': n3, 'discounted_price': n3, 'discount': False}

    
    return price

In [10]:
def extract_review_info(hotel: bs4.element.Tag) -> Dict[str, str]:
    rating_div = hotel.find('div', {'data-testid': 'review-score'})
    if not rating_div:
        return {
            'review_score': None,
            'review_title': None,
            'number_of_reviews': None
        }
    review_divs = rating_div.find_all('div')
    
    return {
        'review_score': review_divs[0].get_text(separator=':)', strip=True).split(':)')[1],
        'review_title': review_divs[3].get_text(strip=True),
        'number_of_reviews': review_divs[4].get_text(separator=' ', strip=True).split(' ')[0]
    }


In [11]:
def extract_star_rating(hotel: bs4.element.Tag) -> Dict[str, Optional[int]]:
    star_rating  = {'star_rating': None, 'is_apartment': None}
    
    star_rating_div = hotel.find('div', {'data-testid': 'rating-stars'})
    square_rating_div = hotel.find('div', {'data-testid': 'rating-squares'})
    if star_rating_div:
        star_rating =  {'star_rating': len(star_rating_div.find_all('svg')), 'is_apartment': False }
    elif square_rating_div:
        star_rating =  {'star_rating': len(square_rating_div.find_all('svg')), 'is_apartment': True }
    
    return star_rating


In [12]:
def extract_neighborhood(hotel: bs4.element.Tag) -> Dict[str, str]:
    neighborhood_span = hotel.find('span', {'data-testid' : 'address'})
    if neighborhood_span: 
        return {'neighborhood': neighborhood_span.get_text(strip=True)}
    else: 
        return {'neighborhood': None}

In [13]:
import re
def extract_kms_from_centre(hotel: bs4.element.Tag) -> Dict[str, Optional[float]]:
    distance_div = hotel.find('span', {'data-testid': 'distance'})
    if  not distance_div:
        return {'kms_from_centre': None}
        
    distance = distance_div.get_text(strip=True)
    if not distance:
        return {'kms_from_centre': None}
        
    from_centre = re.search(r'([\d]+(?:\.\d+)?)\s*km\s+from\s+centre', distance)
    if not from_centre:
        return {'kms_from_centre': None}

    return {'kms_from_centre': float(from_centre.group(1))}
    

In [14]:
def extract_location_score(hotel: bs4.element.Tag) -> Dict[str, Optional[float]]:
    location_link = hotel.find('a', {'data-testid': 'secondary-review-score-link'})
    
    if not location_link:
        return {'location_score': None}
    
    location_score_match = re.search(r'Scored\s(\d+\.\d+)', location_link['aria-label'])
    
    if not location_score_match:
        return {'location_score': None}
    
    return {'location_score': float(location_score_match.group(1))}


In [15]:
from lxml import etree

def extract_sleeping_arrangements(hotel: bs4.element.Tag) -> Dict[str, Optional[str]]:
    recommended_units_div = hotel.find('div', {'data-testid': 'recommended-units'})
    
    sleeping_arrangements = {}
    
    hotel_tree = etree.HTML(str(hotel))
    bed_type_xpath = hotel_tree.xpath('//div[@data-testid="recommended-units"]//ul/li/div[2]/text()')
    
    if '•' in recommended_units_div.get_text(strip=True):
        if(len(bed_type_xpath) > 0):
            sleeping_arrangements['bed_type'] = bed_type_xpath[0].text.strip()
    else:
        sleeping_arrangements['bed_type'] = (recommended_units_div and 
                                recommended_units_div.ul and 
                                recommended_units_div.ul.li and 
                                recommended_units_div.ul.li.div and 
                                recommended_units_div.ul.li.div.div and 
                                recommended_units_div.ul.li.div.div.get_text(strip=True)) or None
        
    sleeping_arrangements['room_type'] = (recommended_units_div and 
                               recommended_units_div.h4 and 
                               recommended_units_div.h4.get_text(strip=True)) or None
    
    return sleeping_arrangements


In [16]:
def extract_breakfast_included(hotel: bs4.element.Tag) -> Dict[str, bool]:
    
    if 'breakfast included' in hotel.get_text(strip=True).lower():
        return {'breakfast_included': True}
    else:
        return {'breakfast_included': False}


In [17]:
def extract_payment_policy(hotel: bs4.element.Tag) -> Dict[str, str]:
    payment_policy = {}
    
    payment_policy['free_cancellation'] = True if hotel.find('span', {'data-testid': 'cancellation-policy-icon'}) else False
    payment_policy['prepayment_needed'] = False if hotel.find('span', {'data-testid': 'prepayment-policy-icon'}) else True 
    
    return payment_policy


In [18]:
def extract_hotel_info(hotel: bs4.element.Tag) -> Dict[str, Any]:
    hotel_info = {}
    
    extraction_functions = [
        extract_name,
        extract_price, 
        extract_review_info,
        extract_star_rating,
        extract_neighborhood,
        extract_kms_from_centre,
        extract_location_score,
        extract_sleeping_arrangements,
        extract_breakfast_included,
        extract_payment_policy
        
    ]
    
    for func in extraction_functions:
        hotel_info.update(func(hotel))

    return hotel_info

In [19]:
import pandas as pd 

def get_hotels_dataframe(hotels: List[bs4.element.Tag]) -> pd.DataFrame:
    hotels_info = [extract_hotel_info(hotel) for hotel in hotels]
    return pd.DataFrame(hotels_info)

In [20]:
hotels= get_hotels(init_driver(), build_url(1,1))

In [21]:
hotels_df = get_hotels_dataframe(hotels)

In [22]:
hotels_df

,name,original_price,discounted_price,discount,review_score,review_title,number_of_reviews,star_rating,is_apartment,neighborhood,kms_from_centre,location_score,bed_type,room_type,breakfast_included,free_cancellation,prepayment_needed
0,Riverside Tower Hotel,140,140,False,8.5,Very good,"2,282",2.0,False,"Upper West Side, New York",None,NaN,1 double bed,Accessible Double Room,False,False,True
1,"The Mayfair Hotel Times Square, Ascend Hotel C...",141,141,False,8.6,Fabulous,"1,906",3.0,False,"Manhattan, New York",None,9.6,1 double bed,Deluxe Double Room - Non Smoking,False,False,True
2,M Social Hotel Times Square New York,220,220,False,8.4,Very good,"11,796",4.0,False,"Manhattan, New York",None,9.6,2 double beds,Social City View Doubles,False,False,True
3,"The Cloud One New York-Downtown, by the Motel ...",163,163,False,8.6,Fabulous,"6,377",4.0,False,"Manhattan, New York",None,9.4,1 extra-large double bed,Standard King Room,False,False,True
4,Wingate by Wyndham New York Midtown South/5th Ave,155,155,False,8.4,Very good,"2,386",3.0,False,"Manhattan, New York",None,9.5,1 large double bed,Queen Room - Non-Smoking,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Staypineapple, An Artful Hotel, Midtown New York",193,193,False,8.6,Fabulous,"1,643",4.0,False,"Hell's Kitchen, New York",None,NaN,1 extra-large double bed,Celebrity King,False,False,True
96,"Holiday Inn Express New York City Chelsea, an ...",215,215,False,7.8,Good,"3,627",3.0,False,"Chelsea, New York",None,NaN,1 extra-large double bed,Standard King Room with Mobility Accessible Tub,True,False,True
97,Hilton Garden Inn NYC Financial Center/Manhatt...,186,186,False,8.3,Very good,"5,048",3.0,False,"Manhattan, New York",None,NaN,1 extra-large double bed,King Bed with City View,False,True,False
98,DoubleTree by Hilton New York Midtown Fifth Ave,266,266,False,7.6,Good,"1,707",4.0,False,"Manhattan, New York",None,9.4,1 large double bed,Queen Room,False,False,True


In [23]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from datetime import datetime
import os
import math

SNAPSHOT_DATE = datetime.today().strftime('%Y-%m-%d')

number_of_cores = os.cpu_count()
max_workers = math.ceil(number_of_cores/3)


def scrape_hotels_thread(TTT, LOS):
    """Scrape hotel data using a shared WebDriver with multiple tabs."""
    print(f'scraping TTT={TTT} / 30, LOS={LOS} / 5')
    try:
        url = build_url(TTT, LOS)

        driver = init_driver(headless=True)  
        # Open a new tab
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])  # Switch to new tab
        driver.get(url)

        hotels = get_hotels(driver, url)
        hotels_df = get_hotels_dataframe(hotels)
        hotels_df['time_to_travel'] = TTT
        hotels_df['length_of_stay'] = LOS
        hotels_df['snapshot_date'] = SNAPSHOT_DATE

        driver.close()  # Close tab after scraping
        driver.switch_to.window(driver.window_handles[0])  # Switch back to main tab

        return hotels_df
    
    except Exception as e:
        print(f"Error scraping TTT={TTT}, LOS={LOS}: {e}")
        return pd.DataFrame()  # Return empty DataFrame in case of failure
    finally:
        if driver:
            driver.quit()

all_combinations = [(TTT, LOS) for TTT in range(1, 31) for LOS in range(1, 6)]

# Run threads, each using a new tab in the same browser
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    results = list(executor.map(lambda args: scrape_hotels_thread(*args), all_combinations))
    df = pd.concat(results, ignore_index=True)
    df.to_csv(f'booking_snapshot_{SNAPSHOT_DATE}.csv', index=False)

scraping TTT=1 / 30, LOS=1 / 5scraping TTT=1 / 30, LOS=2 / 5

scraping TTT=1 / 30, LOS=3 / 5
scraping TTT=1 / 30, LOS=4 / 5
scraping TTT=1 / 30, LOS=5 / 5
scraping TTT=2 / 30, LOS=1 / 5
scraping TTT=2 / 30, LOS=2 / 5
scraping TTT=2 / 30, LOS=3 / 5
scraping TTT=2 / 30, LOS=4 / 5
scraping TTT=2 / 30, LOS=5 / 5
scraping TTT=3 / 30, LOS=1 / 5
scraping TTT=3 / 30, LOS=2 / 5
scraping TTT=3 / 30, LOS=3 / 5
scraping TTT=3 / 30, LOS=4 / 5
scraping TTT=3 / 30, LOS=5 / 5
scraping TTT=4 / 30, LOS=1 / 5
scraping TTT=4 / 30, LOS=2 / 5
scraping TTT=4 / 30, LOS=3 / 5
scraping TTT=4 / 30, LOS=4 / 5
scraping TTT=4 / 30, LOS=5 / 5
scraping TTT=5 / 30, LOS=1 / 5
scraping TTT=5 / 30, LOS=2 / 5
scraping TTT=5 / 30, LOS=3 / 5
scraping TTT=5 / 30, LOS=4 / 5
scraping TTT=5 / 30, LOS=5 / 5
scraping TTT=6 / 30, LOS=1 / 5
scraping TTT=6 / 30, LOS=2 / 5
scraping TTT=6 / 30, LOS=3 / 5
scraping TTT=6 / 30, LOS=4 / 5
scraping TTT=6 / 30, LOS=5 / 5
scraping TTT=7 / 30, LOS=1 / 5
scraping TTT=7 / 30, LOS=2 / 5
scraping

C:\Users\HamamaHome\AppData\Local\Temp\ipykernel_12232\4259584399.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(results, ignore_index=True)


In [26]:


df1 = pd.read_csv('./booking_snapshot_2025-02-02.csv')
df2 = pd.read_csv('./booking_snapshot_2025-02-03.csv')
df3 = pd.read_csv('./booking_snapshot_2025-02-04.csv')

combined_df = pd.concat([df1,df2,df3], ignore_index=True)

In [27]:
combined_df.to_csv('./bookings.csv')